In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Project/'
#%pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1XSIbHWFizvrGifGPTwtsmagK2AS8ZE7n/Project


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
np.random.seed(682)
tf.random.set_seed(682)

In [ ]:
class Distiller(keras.Model):
  
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
teacher = tf.keras.models.load_model('/saved_models/resnet50cifar')

In [ ]:
# Import necessary components to build LeNet
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.regularizers import l2

def lenet_model(img_shape=(32, 32, 3), n_classes=10, l2_reg=0.,
	weights=None):

	# Initialize model
	lenet = Sequential()

	# 2 sets of CRP (Convolution, RELU, Pooling)
	lenet.add(Conv2D(20, (5, 5), padding="same",
		input_shape=img_shape, kernel_regularizer=l2(l2_reg)))
	lenet.add(Activation("relu"))
	lenet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

	lenet.add(Conv2D(50, (5, 5), padding="same",
		kernel_regularizer=l2(l2_reg)))
	lenet.add(Activation("relu"))
	lenet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

	# Fully connected layers (w/ RELU)
	lenet.add(Flatten())
	lenet.add(Dense(500, kernel_regularizer=l2(l2_reg)))
	lenet.add(Activation("relu"))

	# Softmax (for classification)
	lenet.add(Dense(n_classes, kernel_regularizer=l2(l2_reg)))
	# lenet.add(Activation("softmax"))

	if weights is not None:
		lenet.load_weights(weights)

	# Return the constructed network
	return lenet

In [ ]:
# Create the student
student = lenet_model()

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64

(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 102s 59ms/step - sparse_categorical_accuracy: 0.3256 - student_loss: 2.3490 - distillation_loss: 0.0283
Epoch 2/5
1563/1563 [==============================] - 91s 58ms/step - sparse_categorical_accuracy: 0.5073 - student_loss: 1.4052 - distillation_loss: 0.0094
Epoch 3/5
1563/1563 [==============================] - 91s 58ms/step - sparse_categorical_accuracy: 0.5868 - student_loss: 1.2121 - distillation_loss: 0.0110
Epoch 4/5
1563/1563 [==============================] - 91s 58ms/step - sparse_categorical_accuracy: 0.6367 - student_loss: 1.0784 - distillation_loss: 0.0124
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.5802 - student_loss: 1.2171


[0.5802000164985657, 1.4431142807006836]

In [ ]:
# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

313/313 [==============================] - 1s 5ms/step - sparse_categorical_accuracy: 0.5802 - student_loss: 1.2171


[0.5802000164985657, 1.4431142807006836]

In [ ]:
student.save('/saved_models/ts_2')

student_test = tf.keras.models.load_model('/saved_models/ts_2')

# Initialize and compile distiller
distiller_test = Distiller(student=student_test, teacher=teacher)
metric_student = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')

distiller_test.compile (
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/ts_2/assets
313/313 [==============================] - 2s 5ms/step - s1_acc: 0.5802 - student_loss: 1.2171


[0.5802000164985657, 1.4431142807006836]

In [ ]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.9707 - sparse_categorical_accuracy: 0.4144
Epoch 2/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.3044 - sparse_categorical_accuracy: 0.5410
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 1.1469 - sparse_categorical_accuracy: 0.5978
Epoch 4/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0188 - sparse_categorical_accuracy: 0.6475
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 1.3955 - sparse_categorical_accuracy: 0.5650


[1.3955481052398682, 0.5649999976158142]

In [ ]:
student_scratch.save('/saved_models/scratch_2')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/scratch_2/assets


In [ ]:
student_scratch_test = tf.keras.models.load_model('/saved_models/scratch_2')

# Train student as doen usually
student_scratch_test.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch_test.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.3955 - sparse_categorical_accuracy: 0.5650


[1.3955481052398682, 0.5649999976158142]